# CS/ECE/ISyE 524 - Spring 2017 - Homework 7 - Solutions
### Prepared by: Laurent Lessard

# 1. Thrift store

How should you make change for 99 cents if the goal is to minimize the total weight of the coins used? The following table shows the weight of each type of coin. You may use any number of each type of coin.

| Type of coin | penny | nickel | dime | quarter |
|:------------:|:-----:|:------:|:----:|:-------:|
|Weight (grams)| 2.500 |  5.000 |2.268 |  5.670  |

In [3]:
coinType = [ :penny, :nickel, :dime, :quarter ]

# problem data (we'll use a dictionary here)
weights = Dict( :penny => 2.500, :nickel => 5.000, :dime => 2.268, :quarter => 5.670 )
values  = Dict( :penny => 1    , :nickel => 5    , :dime => 10   , :quarter => 25    )

using JuMP, Cbc, Gurobi, Mosek, GLPKMathProgInterface
m = Model(solver = MosekSolver())

@variable(m, x[coinType] >= 0, Int)
@constraint(m, sum( x[i] * values[i] for i in coinType ) == 99 )
@objective(m, Min, sum( x[i] * weights[i] for i in coinType ) )
solve(m)

println(getvalue(x))
println(getobjectivevalue(m))

x: 1 dimensions:
[  penny] = 4.0
[ nickel] = 0.0
[   dime] = 2.0
[quarter] = 3.0
31.546


# 2. Comquat Computers

Comquat owns four production plants at which personal computers are produced. Comquat can sell up to 20,000 computers per year at a price of \$3,500 per computer. For each plant the production capacity, cost per computer, and fixed cost of operating the plant for a year are given below. Determine how Comquat can maximize its yearly profit from computer production.

| Plant | Production capacity | Plant fixed cost (\$ Million) | Cost per computer (\$) |
|:-----:|:-------------------:|:-----------------------------:|:----------------------:|
| 1     | 10,000              |   9                           |  1,000                 |
| 2     |  8,000              |   5                           |  1,700                 |
| 3     |  9,000              |   3                           |  2,300                 |
| 4     |  6,000              |   1                           |  2,900                 |

In [18]:
using JuMP, Cbc
m = Model(solver = CbcSolver())

price     = 3.5e3                       # price at which we can sell a single computer (regardless of where it is produced)
cap       = 20000                       # maximum computers that can be sold per year

capacity  = [10000, 8000, 9000, 6000]   # production capacity for each plant
fixedcost = [9e6, 5e6, 3e6, 1e6]        # fixed cost for each plant
itemcost  = [1e3, 1.7e3, 2.3e3, 2.9e3]  # cost per computer for each factory

@variable(m, z[1:4], Bin)   # indicator variables: is plant i on or off?
@variable(m, x[1:4] >= 0)   # number of computers produced at each of the plants

# production constraints on each plant
@constraint(m, x .<= capacity)

# total production capacity constraint
@constraint(m, sum(x) <= cap)

# logic constraint: if z[i]=0 then we must have x[i]=0
@constraint(m, x .<= capacity .* z)

# expenses (item costs + fixed costs)
@expression(m, expenses, sum( itemcost[i]*x[i] for i in 1:4) + sum( fixedcost[i]*z[i] for i in 1:4) )

# revenue (from selling items)
@expression(m, revenue, price*sum(x))

# MAXIMIZE PROFIT (revenue - expenses)
@objective(m, Max, revenue - expenses)

solve(m)
println("The maximal profit is: ", getobjectivevalue(m))
println("Here is a tally of factory number, whether it's open or not, and how many computers it should produce.")
getvalue([1:4 z x])

The maximal profit is: 2.56e7
Here is a tally of factory number, whether it's open or not, and how many computers it should produce.


4×3 Array{Float64,2}:
 1.0  1.0  10000.0
 2.0  1.0   8000.0
 3.0  0.0      0.0
 4.0  1.0   2000.0

# 3. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

In [19]:
using JuMP, Cbc
m = Model(solver = CbcSolver())

lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return

@variable(m, x[1:6] >= 0)             # amount invested in each option
@variable(m, z[1:6], Bin)             # indicator variable: do we invest in option i or not?

@constraint(m, sum(x) <= 80)     # total investment limit

# option 5 is no more than combined amounts from options 2, 4, and 6
@constraint(m, x[5] <= x[2] + x[4] + x[6])

# if there is an investment in option 3, we must invest in option 6.
# in other words: if z[3] = 1, then z[6] = 1.
@constraint(m, z[3] <= z[6])

# variable lower bounds (if z=0 then x=0. if z=1 then low <= x <= hi)
for i = 1:6
    @constraint(m, lows[i]*z[i] <= x[i]  )
    @constraint(m, x[i] <= highs[i]*z[i] )
end

@expression(m, expectedReturn, sum( x[i] * ret[i]/100 for i = 1:6) )

@objective(m, Max, expectedReturn)

solve(m)
println("The maximum return on investment (net profit) is: \$", getobjectivevalue(m), " million.")
println("This is an average return of: ", 100*getobjectivevalue(m)/getvalue(sum(x)), " %")
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
getvalue([1:6 z x])

The maximum return on investment (net profit) is: $13.5 million.
This is an average return of: 16.875 %
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).


6×3 Array{Float64,2}:
 1.0  0.0   0.0
 2.0  0.0   0.0
 3.0  1.0  35.0
 4.0  1.0   5.0
 5.0  1.0  22.5
 6.0  1.0  17.5

## 4. Lights Out

 In Tiger Electronic's handheld solitaire game Lights Out, the player strives to turn out all 25 lights that make up a $5\times 5$ grid of cells.  On each turn, the player is allowed to click on any one cell. Clicking on a cell activates a switch that causes the states of the cell and its (edge) neighbors to change from on to off, or from off to on. Corner cells are considered to have 2 neighbors, edge cells to have three, and interior cells to have four. Find a way to turn out all the lights in as few turns as possible, starting from the state where all lights are on.

**Hints:** The order in which the cells are clicked doesn't matter, and there is no need to click any cell more than once.

In [4]:
# there are many ways to model the logic constraints required here.
# We will solve the problem in one possible way by using an integer variable
# to model the parity constraint of clicking a button an even/odd number of times.

N = 5               # size of the grid
Xinit = ones(N,N)   # initial state (every light is on)


using JuMP, Cbc
m = Model(solver = CbcSolver())
                                         # X[i,j] is 1 if we click cell (i,j) and 0 otherwise.
@variable(m, X[1:N+2,1:N+2], Bin)        # NOTE: we add an extra border of cells around the grid to eliminate special cases
                                         # so the effective size of the grid is now (N+2)×(N+2)
@variable(m, 0 <= Z[1:N,1:N] <= 2, Int)  # one integer-valued variable per cell

# each cell must get affected 0, 2, or 4 times if Xinit=0 and 1, 3, or 5 times if Xinit=1
# we model this as 2*z), which can take on values of 0, 2, 4 depending on choice of z ∈ {0,1,2}
for i = 2:N+1
    for j = 2:N+1
        @constraint(m, X[i,j]+X[i-1,j]+X[i+1,j]+X[i,j-1]+X[i,j+1] == Xinit[i-1,j-1]+2Z[i-1,j-1,1] )
    end
end

# border cells are pegged at zero
for i = 1:N+2
    @constraint(m, X[1,i] == 0)
    @constraint(m, X[N+2,i] == 0)
    @constraint(m, X[i,1] == 0)
    @constraint(m, X[i,N+2] == 0)
end

# minimize number of button presses
@objective(m, Min, sum(X))

@time(solve(m))
Xopt = getvalue( X[2:N+1,2:N+1] )
convert(Array{Int64}, Xopt)

# NOTE: the solution is not unique (e.g. you can rotate by 90 deg, mirror,...)

  0.423378 seconds (103 allocations: 26.938 KB)


5×5 Array{Int64,2}:
 1  1  0  0  0
 1  1  0  1  1
 0  0  1  1  1
 0  1  1  1  0
 0  1  1  0  1